<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/BiLSTM%20CRF%20Model%20(ELMO%20Embeddings).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings Bi-LSTM-CRF Model

In [2]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro (2).py


In [3]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-eh60kqwe
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-eh60kqwe
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=6b58adb220d0c348064f2d54d3f0839ae3cfa9adf8abca523f892897bac2a0d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-i18mpf5z/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
!pip install sklearn_crfsuite

In [5]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

Using TensorFlow backend.


In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [7]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words= [w.lower() for w in words]

n_words = len(set(words))
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [8]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 4


In [9]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 9497


In [0]:
def replace_number(word):
  if any(i.isdigit() for i in word):
    return "d"
  else:
    return word

In [11]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [0]:
sentences = [[w[0] for w in s] for s in train]

new_X = []
for seq in sentences:
    new_seq = []
    for i in range(MAX_LEN):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X = new_X

batch_size = 200
div = int(np.floor(len(X)/batch_size))

X = X[:div*batch_size]

In [83]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([4, 3, 5, 3, 3, 3, 5, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]
y = y[:div*batch_size]

## ELMO Embeddings

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from keras.layers.merge import add
from keras.layers import Lambda

In [0]:
sess = tf.Session()
K.set_session(sess)

In [0]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [0]:
max_len = MAX_LEN
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32)
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [104]:
# Model definition

input = Input(shape=(MAX_LEN,),dtype=tf.string)

embedding = Lambda(ElmoEmbedding, output_shape=(None,None,MAX_LEN,1024))(input)

lstm1 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(embedding)

lstm2 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(lstm1)

model_2 = TimeDistributed(Dense(10, activation="relu"))(lstm2)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model(input, out)

W0818 21:33:48.613198 139860942051200 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/deprecation.py:507: calling count_nonzero (from tensorflow.python.ops.math_ops) with axis is deprecated and will be removed in a future version.
Instructions for updating:
reduction_indices is deprecated, use axis instead


In [105]:
model_2.compile(optimizer='adam', loss=crf_loss,metrics=["acc"])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 124)               0         
_________________________________________________________________
lambda_11 (Lambda)           (None, None, None, 124, 1 0         
_________________________________________________________________
bidirectional_20 (Bidirectio (None, None, 128)         557568    
_________________________________________________________________
bidirectional_21 (Bidirectio (None, None, 128)         98816     
_________________________________________________________________
time_distributed_9 (TimeDist (None, None, 10)          1290      
_________________________________________________________________
crf_7 (CRF)                  (None, None, 6)           114       
Total params: 657,788
Trainable params: 657,788
Non-trainable params: 0
_________________________________________________________________


In [106]:
%%time
BATCH_SIZE = 200
EPOCHS=1

history = model_2.fit(np.array(X), np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

Train on 11200 samples, validate on 2800 samples
Epoch 1/1
11200/11200 [==============================] - 312s 28ms/step - loss: 0.3940 - acc: 0.8880 - val_loss: 0.0708 - val_acc: 0.9814
CPU times: user 2min 29s, sys: 16.3 s, total: 2min 45s
Wall time: 5min 15s


## Predict on Training Set

In [0]:
pred = model_2.predict(np.array(X),verbose =1)

In [123]:
# TRain Eval
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         LOC       0.31      0.00      0.00      8261
        MISC       0.00      0.00      0.00      4587
           O       0.85      1.00      0.92    169339
         ORG       0.40      0.04      0.07     10011
         PAD       1.00      1.00      1.00   1532754
         PER       0.77      0.13      0.23     11048

    accuracy                           0.98   1736000
   macro avg       0.56      0.36      0.37   1736000
weighted avg       0.97      0.98      0.97   1736000

0.9740227996599867


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [124]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         LOC       0.31      0.00      0.00      8261
         PER       0.77      0.13      0.23     11048
         ORG       0.40      0.04      0.07     10011
        MISC       0.00      0.00      0.00      4587

   micro avg       0.64      0.05      0.10     33907
   macro avg       0.37      0.04      0.07     33907
weighted avg       0.45      0.05      0.10     33907

0.09516099880154641


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Predict on Test Set

In [0]:
# Convert each sentence from list of Token to list of word_index
X_test = [[word2idx[w[0].lower()] for w in s] for s in test]

# Padding each sentence to have the same lenght
X_test = pad_sequences(maxlen=MAX_LEN, sequences=X_test, padding="post", value=word2idx["PAD"])
X_test

# Convert Tag/Label to tag_index
y_test = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y_test = pad_sequences(maxlen=MAX_LEN, sequences=y_test, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y_test = [to_categorical(i, num_classes=n_tags+1) for i in y_test]

In [0]:
pred = model_2.predict(X_test, verbose=1)

3453/3453 [==============================] - 47s 14ms/step


In [0]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_test, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)


              precision    recall  f1-score   support

        MISC       0.71      0.70      0.70       918
         LOC       0.87      0.89      0.88      1925
         ORG       0.87      0.74      0.80      2496
         PER       0.96      0.94      0.95      2773

   micro avg       0.88      0.84      0.86      8112
   macro avg       0.85      0.81      0.83      8112
weighted avg       0.88      0.84      0.86      8112

0.8572302440182686
